# Carga de Datos

## Librerías que se utilizarán

In [6]:
import os
import pandas as pd
import re

### Con el siguiente bloque de código concatemos los path de los archivos csv para después crear un dataframe

In [42]:
#contaminantes_catalogos
path_catalogos = r'/Users/sebastian/PycharmProjects/RegresionLinealSimple/data/contaminantes_catalogos/'
files_catalogos = os.listdir(path_catalogos)
file_list_catalogos = [files_catalogos for files_catalogos in files_catalogos if files_catalogos[:3] == 'cat']
file_list_catalogos

['cat_unidades.csv', 'cat_estacion.csv', 'cat_parametros.csv']

In [60]:
path_csv_cat = []
#path_csv_cat = [os.path.join(path_catalogos, n) for n in file_list_catalogos]
i =0
for file in file_list_catalogos:
    print(file)
    path_csv_cat.append(path_catalogos+file)
    i+=1

path_csv_cat
path_csv_cat
df = pd.read_csv(path_csv_cat[1], sep=',',  encoding='latin-1')
df.head()
#df.info

cat_unidades.csv
cat_estacion.csv
cat_parametros.csv


,Catalogo de estaciones,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,cve_estac,nom_estac,longitud,latitud,alt,obs_estac,id_station
1,ACO,Acolman,-98.912003,19.635501,2198,NaN,484150020109
2,AJU,Ajusco,-99.162611,19.154286,2942,NaN,484090120400
3,AJM,Ajusco Medio,-99.207744,19.272161,2548,NaN,484090120609
4,ARA,Aragón,-99.074549,19.470218,2200,Finalizó operación en 2010,484090050301


In [11]:
#Contaminantes_CDMX
path_cdmx = '/Users/sebastian/PycharmProjects/RegresionLinealSimple/data/contaminantes_CDMX'
files_cdmx = os.listdir(path_cdmx)
file_list_cdmx = [files_cdmx for files_cdmx in files_cdmx if files_cdmx[:3] == 'con']
file_list_cdmx

['contaminantes_2022.csv',
 'contaminantes_2021.CSV',
 'contaminantes_2020.CSV',
 'contaminantes_2018.CSV',
 'contaminantes_2019.CSV']